# Proof of Concept: Question/Answering System over SQL Data with LLM

## 1. Introduction
- **Objective**: Develop a PoC system that enables natural language querying of SQL databases, leveraging LLM capabilities.
- **Overview**: The system is implemented using DuckDB, LangChain, and ChatOpenAI's "gpt-4o-mini" model.

This Jupyter notebook demonstrates a Proof of Concept (PoC) implementation of a Question/Answering system over SQL data, powered by a ChatOpenAI "gpt-4o-mini" model integrated with LangChain.

### Key Features:
1. **Data Loading and SQLite Database Setup**:
   - Converts a provided Excel dataset into a DuckDB database.
   - Validates the database with example SQL queries.

2. **LLM-Based Q&A System**:
   - Implements a LangGraph React agent to translate natural language questions into SQL, executes SQL queries and returns natural language responses.

3. **Testing and Demonstration**:
   - Tests the system with example natural language questions.

4. **Interactive Chat-Like UI**:
   - Provides an `ipywidgets`-based interface for real-time testing.
   - Enables users to interact with the system through natural language questions. [at the Bottom of the Notebook]

### Purpose:
This PoC solution enables users to perform data profiling, analytics, and quality assessments by asking simple questions and receiving concise, actionable insights.

In [1]:
!pip install pandas==2.2.2
!pip install SQLAlchemy==2.0.36
!pip install duckdb==1.1.3
!pip install duckdb-engine==0.14.0
!pip install openai==1.58.1
!pip install langchain-openai==0.2.14
!pip install langchain-community==0.3.13
!pip install langgraph==0.2.60
!pip install ipywidgets==8.1.5

Defaulting to user installation because normal site-packages is not writeable
    sys-platform (=="darwin") ; extra == 'objc'
                 ~^
Defaulting to user installation because normal site-packages is not writeable
    sys-platform (=="darwin") ; extra == 'objc'
                 ~^
Defaulting to user installation because normal site-packages is not writeable
    sys-platform (=="darwin") ; extra == 'objc'
                 ~^
Defaulting to user installation because normal site-packages is not writeable
    sys-platform (=="darwin") ; extra == 'objc'
                 ~^
Defaulting to user installation because normal site-packages is not writeable
    sys-platform (=="darwin") ; extra == 'objc'
                 ~^
Defaulting to user installation because normal site-packages is not writeable


    sys-platform (=="darwin") ; extra == 'objc'
                 ~^
Defaulting to user installation because normal site-packages is not writeable


    sys-platform (=="darwin") ; extra == 'objc'
                 ~^
Defaulting to user installation because normal site-packages is not writeable


    sys-platform (=="darwin") ; extra == 'objc'
                 ~^
Defaulting to user installation because normal site-packages is not writeable
    sys-platform (=="darwin") ; extra == 'objc'
                 ~^


In [2]:
# Import necessary libraries
import os
import warnings
import re
from datetime import datetime
import pprint
import pandas as pd
import itertools
import base64
import markdown

from IPython.display import HTML, display, Markdown
from ipywidgets import widgets

import duckdb

from sqlalchemy import create_engine, event, text
from sqlalchemy import inspect
from sqlalchemy.engine import Connection
from sqlalchemy.exc import DBAPIError

from langchain_openai import ChatOpenAI
from langchain_community.utilities import SQLDatabase
from langchain_core.messages import HumanMessage
from langchain_core.prompts import PromptTemplate, FewShotPromptTemplate
from langchain.agents.agent_toolkits import SQLDatabaseToolkit
from langchain.memory import ConversationBufferMemory
from langgraph.prebuilt import create_react_agent
from langgraph.checkpoint.memory import MemorySaver

In [3]:
warnings.filterwarnings("ignore")

In [4]:
# Set the OpenAI API key in the environment variables.
# Replace <YOUR_KEY_HERE> with the actual API key string.
os.environ["OPENAI_API_KEY"] = "<YOUR_KEY_HERE>"

## 2. Data Loading and Database Setup

### 2.1 Loading Data from Excel file

In [5]:
# Read data from the specified Excel file into a DataFrame
file_path = './data/Actual_Accounts.xlsx'
df = pd.read_excel(file_path, index_col=0)

FileNotFoundError: [Errno 2] No such file or directory: './data/Actual_Accounts.xlsx'

In [ ]:
# Display the first few rows of the DataFrame to verify data loading.
df.head()

In [ ]:
# Inspect the data types of each column in the DataFrame.
df.dtypes

In [ ]:
# Generate descriptive statistics for the numeric columns.
df.describe()

### 2.2 Creating the DuckDB Database

In [ ]:
# Specify the file path for the DuckDB database
database_path = "HomeTask_PwC.duckdb"

# Create a persistent DuckDB engine
engine = create_engine(f"duckdb:///{database_path}")

# # Create DuckDB in-memory engine
# engine = create_engine("duckdb:///:memory:", poolclass=None, future=True)

# # Open a single shared connection
# conn = engine.connect()

In [ ]:
base_name = os.path.splitext(os.path.basename(file_path))[0]

In [ ]:
def get_table_name(file_path):
    # Extract the base file name without extension
    base_name = os.path.splitext(os.path.basename(file_path))[0]
    
    # Replace one or more non-alphanumeric characters with a single underscore
    sanitized_name = re.sub(r'[^a-zA-Z0-9]+', '_', base_name)
    
    # Remove leading and trailing underscores
    sanitized_name = sanitized_name.strip('_')
    
    # Ensure the table name starts with a letter or underscore
    if not sanitized_name[0].isalpha():
        sanitized_name = f"table_{sanitized_name}"
    
    return sanitized_name

In [ ]:
table_name = get_table_name(file_path)

In [ ]:
table_name

In [ ]:
# Attempt to write the DataFrame into the specified SQL table. 
# If the table already exists, replace it. Log success or display any encountered error.
try:
    df.to_sql(table_name, con=engine, if_exists="replace", index=True)
    print(f"Data successfully written to the table: {table_name}")
except Exception as e:
    print(f"An error occurred: {e}")

In [ ]:
# Define and enforce a rule prohibiting DML statements (DROP, INSERT, UPDATE, DELETE, MERGE)
# by listening for execution events.
# If a forbidden statement is detected, raise an error before cursor execution.

# List of forbidden keywords
FORBIDDEN_KEYWORDS = ["drop", "insert", "update", "delete", "merge"]

def validate_no_dml(engine, clause, multiparams, params):
    # Extract the SQL statement
    statement = str(clause).strip().lower()

    # Check if the statement starts with forbidden keywords
    if any(statement.startswith(keyword) for keyword in FORBIDDEN_KEYWORDS):
        raise DBAPIError("Nice try ;), but DML statements are not allowed (INSERT, UPDATE, DELETE, MERGE).", None, None)

# Attach the event listener to the engine
event.listen(engine, "before_execute", validate_no_dml)

### 2.3 Example SQL Queries

In [ ]:
# Execute a quick test query to show all available tables
query = 'SHOW TABLES;'
engine_test_df = pd.read_sql(query, con=engine)
engine_test_df.head()

In [ ]:
# Execute a quick test query to fetch the first 5 records from the new table
query = f"SELECT * FROM {table_name} LIMIT 5"
engine_test_df = pd.read_sql(query, con=engine)
engine_test_df.head()

In [ ]:
# Execute a quick test query to fetch the maximum absolute transaction value from the specified table, 
query = f'SELECT MAX(ABS("Transaction Value")) AS max_abs_trx_value FROM {table_name};'
engine_test_df = pd.read_sql(query, con=engine)
engine_test_df.head()

In [ ]:
# Execute a quick test query to count how many rows have null and non-null values in the [Transaction Value] column, 
query = f"""
    SELECT 
        COUNT(*) FILTER (WHERE "Transaction Value" IS NULL) AS null_count,
        COUNT(*) FILTER (WHERE "Transaction Value" IS NOT NULL) AS non_null_count
    FROM {table_name};
"""

engine_test_df = pd.read_sql(query, con=engine)
engine_test_df.head()

In [ ]:
# Execute a quick test query to count how many observations in the [Fiscal Year.1] column are null and non-null
query = f"""
    SELECT 
        COUNT(*) AS total_count,
        COUNT("Fiscal Year.1") AS non_null_count,
        COUNT(*) - COUNT("Fiscal Year.1") AS null_count
    FROM {table_name};
"""

engine_test_df = pd.read_sql(query, con=engine)
engine_test_df.head()

In [ ]:
# Identify outliers in the [Transaction Value] column based on the 1.5 IQR rule.
# Calculates Q1, Q3, and IQR, then counts transactions below Lower_Bound, above Upper_Bound, 
# and the total number of outliers.

column_name = "Transaction Value"

query = f"""
    WITH Stats AS (
        SELECT 
            QUANTILE("{column_name}", 0.25) AS Q1,
            QUANTILE("{column_name}", 0.75) AS Q3
        FROM {table_name}
    ),
    OutlierBounds AS (
        SELECT 
            Q1,
            Q3,
            (Q3 - Q1) AS IQR,
            Q1 - 1.5 * (Q3 - Q1) AS Lower_Bound,
            Q3 + 1.5 * (Q3 - Q1) AS Upper_Bound
        FROM Stats
    ),
    Outliers AS (
        SELECT 
            COUNT(*) AS Total_Transactions,
            SUM(CASE WHEN "{column_name}" < Lower_Bound THEN 1 ELSE 0 END) AS Below_Lower_Bound,
            SUM(CASE WHEN "{column_name}" > Upper_Bound THEN 1 ELSE 0 END) AS Above_Upper_Bound
        FROM {table_name}, OutlierBounds
    )
    SELECT 
        Below_Lower_Bound,
        Above_Upper_Bound,
        (Below_Lower_Bound + Above_Upper_Bound) AS Total_Outliers
    FROM Outliers;
"""

In [ ]:
%%time
engine_test_df = pd.read_sql(query, con=engine)

In [ ]:
engine_test_df.head()

In [ ]:
column_name = "Transaction Value"
query = f'SELECT QUANTILE("{column_name}", 0.75) AS Q3 FROM {table_name};'

engine_test_df = pd.read_sql(query, con=engine)
engine_test_df.head()

## 3. Building the Question/Answering LLM System

In [ ]:
# Check ables detected by SQLAlchemy
inspector = inspect(engine)
tables = inspector.get_table_names()
print("Tables detected by SQLAlchemy:", tables)

In [ ]:
# Initialize an langchain SQLDatabase instance with the existing engine to handle queries.
sql_database = SQLDatabase(engine, include_tables=tables)

In [ ]:
# Now, let's check usable tables
usable_table_names = sql_database.get_usable_table_names()
print(usable_table_names)

In [ ]:
# Create a multi-line prompt template for ChatGPT using the COSTAR framework.
# Include additional instructions and a few-shot example section 
# to guide ChatGPT in responding with concise, SQL-driven insights.

template = '''
Context: The department responsible for data quality management requires a solution for data profiling to allow non-technical users to ask simple questions about their data, such as identifying empty fields or detecting outliers. The goal is to enable interaction with data through plain language queries and return concise, actionable answers based on SQL queries executed on the database.
Objective: Provide accurate, brief, and clear answers to user queries regarding data analytics and quality, utilizing SQL to generate insights.
Scope: Focus on questions related to data profiling, such as:
 - Counts of missing or empty fields.
 - Detection of outliers.
 - Basic descriptive statistics (e.g., min, max, mode, median, averages, distributions).
Target Audience: Data analysts and non-technical users requiring simplified and accessible insights.
Approach: Design responses to be:
 - SQL-driven, based on direct database queries.
 - Short, straightforward, and easily understood by non-technical users.
 - Written in a professional yet approachable tone.
Result: A clear and concise answers to user questions, ensuring they align with the technical accuracy of SQL-based insights while being accessible to non-technical users.

You are an agent designed to interact with a SQL database.

Given an input question, create a syntactically correct {dialect} query to run, then look at the results of the query and return the answer.
Unless the user specifies a specific number of examples they wish to obtain, always limit your query to at most {top_k} results.
You can order the results by a relevant column to return the most interesting examples in the database.
Never query for all the columns from a specific table, only ask for the relevant columns given the question.
You have access to tools for interacting with the database.
Only use the below tools. Only use the information returned by the below tools to construct your final answer.
You MUST double check your query before executing it. If you get an error while executing a query, rewrite the query and try again.

DO NOT make any DML statements (INSERT, UPDATE, DELETE, DROP, MERGE etc.) to the database.

If requested, Generate an SQL query to detect anomalies or outliers in a dataset using the 1.5 IQR (Interquartile Range) method.

To start you should ALWAYS look at the tables in the database to see what you can query. Do NOT skip this step.
Then you should query the schema of the most relevant tables.

Below are a number of examples of questions and their corresponding SQL queries:

  "input": "List all Transaction Type.",
  "query": "SELECT DISTINCT \"Bus. Transac. Type\" FROM TABLE;"

  "input": "Find all Transactions made in 2005 fiscal year.",
  "query": "SELECT * FROM \"TABLE\" WHERE \"Fiscal Year.1\" = 2005;"

  "input": "Give me all Transactions with maximum value.",
  "query": "SELECT * FROM \"TABLE\" WHERE ABS(\"Transaction Value\") = (SELECT MAX(ABS(\"Transaction Value\")) FROM \"[table_name]\");"

"input": "if there are any outliers present in Transaction Value?", # "Give me outliers in the [Transaction Value] column",
"query": """
    WITH Ordered_Values AS (
        SELECT [Transaction Value]
        FROM TABLE
        ORDER BY [Transaction Value]
    ), 
    Row_Numbers AS (
        SELECT 
            [Transaction Value],
            ROW_NUMBER() OVER (ORDER BY [Transaction Value]) AS RowNum,
            COUNT(*) OVER () AS TotalRows
        FROM Ordered_Values
    ), 
    Quartiles AS (
        SELECT 
            MIN(CASE WHEN RowNum = CAST(TotalRows * 0.25 AS INT) THEN [Transaction Value] END) AS Q1,
            MIN(CASE WHEN RowNum = CAST(TotalRows * 0.75 AS INT) THEN [Transaction Value] END) AS Q3
        FROM Row_Numbers
    ), 
    IQR_Calculations AS (
        SELECT 
            Q1,
            Q3,
            (Q3 - Q1) AS IQR,
            Q1 - 1.5 * (Q3 - Q1) AS Lower_Bound,
            Q3 + 1.5 * (Q3 - Q1) AS Upper_Bound
        FROM Quartiles
    )
    SELECT 
        SUM(CASE WHEN [Transaction Value] < Lower_Bound THEN 1 ELSE 0 END) AS Transactions_Below_Lower_Bound,
        SUM(CASE WHEN [Transaction Value] > Upper_Bound THEN 1 ELSE 0 END) AS Transactions_Above_Upper_Bound,
        SUM(CASE WHEN [Transaction Value] < Lower_Bound OR [Transaction Value] > Upper_Bound THEN 1 ELSE 0 END) AS Total_Outliers
    FROM 
        "[table_name]",
        IQR_Calculations;      
        """

"input": "Give me descriptive/basic statistics for the [column_name]",
"query": """        
    WITH basic_stats AS (
        SELECT
            COUNT("[column_name]") AS count_val,
            MIN("[column_name]")   AS min_val,
            MAX("[column_name]")   AS max_val,
            AVG("[column_name]")   AS mean_val
        FROM "table_name"
    ),
    variance_calc AS (
        SELECT
            -- Sample variance with Bessel's correction
            SUM(
                ("[column_name]" - (SELECT mean_val FROM basic_stats)) *
                ("[column_name]" - (SELECT mean_val FROM basic_stats))
            ) / ((SELECT count_val FROM basic_stats) - 1) AS variance_val
        FROM "table_name"
    ),
    quartiles AS (
        SELECT
            QUANTILE("[column_name]", 0.25) AS Q1,
            QUANTILE("[column_name]", 0.50) AS Median,
            QUANTILE("[column_name]", 0.75) AS Q3
        FROM "[table_name]"
    )
    SELECT
        (SELECT count_val FROM basic_stats) AS "Count",
        (SELECT min_val   FROM basic_stats) AS "Min",
        (SELECT max_val   FROM basic_stats) AS "Max",
        (SELECT mean_val  FROM basic_stats) AS "Mean",
        ROUND(SQRT((SELECT variance_val FROM variance_calc)), 3)       AS "StdDev",   -- optional rounding
        (SELECT Q1 FROM quartiles)         AS Q1,
        (SELECT Median FROM quartiles)     AS Median,
        (SELECT Q3 FROM quartiles)         AS Q3;
"""

  "input": "Count null and non-null values in column [column_name]",
  "query": "SELECT COUNT(*) - COUNT(\"[column_name]\") AS COUNT_NULLS, COUNT(\"[column_name]\") AS COUNT_NOT_NULLS FROM \"[table_name]\";"
'''

In [ ]:
# Create a PromptTemplate instance from the multi-line prompt string.
prompt_template = PromptTemplate.from_template(template)

In [ ]:
# Format the prompt template, providing SQLite as the SQL dialect 
# and specifying top_k, which could represent the number of records to retrieve.

system_message = prompt_template.format(dialect="postgresql", top_k=5)

In [ ]:
# Initialize the ChatOpenAI "gpt-4o-mini" model with a specified temperature
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

# create a toolkit with the given database,
toolkit = SQLDatabaseToolkit(db=sql_database, llm=llm)

# retrieve the associated tools from the toolkit
tools = toolkit.get_tools()

In [ ]:
# We can add "chat memory" to the graph with LangGraph's checkpointer
# to retain the chat context between interactions
memory = MemorySaver()

In [ ]:
# Initialize a prebuilt LangGraph React agent with the formatted state_modifier.
agent_executor = create_react_agent(llm, tools, state_modifier=system_message, checkpointer=memory)

## 4. Testing the Q/A System

In [ ]:
config = {"configurable": {"thread_id": "1"}}

In [ ]:
# Execute a quick test by asking a user question
# The agent_executor will stream responses based on the provided SQL database and prompt template.

question = "what is the 3rd quartile?"

for step in agent_executor.stream(
    {"messages": [{"role": "user", "content": question}]},
    stream_mode="values",
    config=config
):
    step["messages"][-1].pretty_print()

In [ ]:
%%time
# Execute a quick test by asking another user question
# The agent_executor will stream responses based on the provided SQL database and prompt template.

question = "Give me descriptive/basic statistics for the Transaction Value?"

for step in agent_executor.stream(
    {"messages": [{"role": "user", "content": question}]},
    stream_mode="values",
    config=config
):
    step["messages"][-1].pretty_print()

In [ ]:
%%time
# Execute a quick test by asking another user question
# The agent_executor will stream responses based on the provided SQL database and prompt template.

question = "if there are any outliers present in Transaction Value?"

for step in agent_executor.stream(
    {"messages": [{"role": "user", "content": question}]},
    stream_mode="values",
    config=config
):
    step["messages"][-1].pretty_print()

In [ ]:
%%time
# Execute a quick test by asking another user question
# The agent_executor will stream responses based on the provided SQL database and prompt template.

question = "Are there any unusual or extreme values in the transaction amounts?"

for step in agent_executor.stream(
    {"messages": [{"role": "user", "content": question}]},
    stream_mode="values",
    config=config
):
    step["messages"][-1].pretty_print()

## 5. Chat-Like Interactive UI (Demo)

In [ ]:
# Define custom CSS styles to enhance the appearance of a chat-like UI, with stylized message bubbles for user and bot responses

In [ ]:
%%html
<style>
    /* General chat container styling */
    .chat-container {
      max-width: 600px;
      margin: 0 auto;
      font-family: Arial, sans-serif;
    }

    /* Chat message alignment */
    .chat-message {
      display: flex;
      margin-bottom: 15px;
      align-items: flex-start;
    }

    .chat-message-left {
      flex-direction: row;
    }

    .chat-message-right {
      flex-direction: row-reverse;
    }

    /* Metadata container (icons and timestamp) */
    .chat-meta {
      display: flex;
      flex-direction: column;
      align-items: center;
      margin: 0 10px;
    }

    /* Icons for user and bot */
    .icon-person::before {
      content: "🧑"; /* User icon */
      font-size: 30px;
    }

    .icon-robot::before {
      content: "🤔"; /* Bot icon */
      font-size: 30px;
    }

    /* Timestamp styling */
    .datetime {
      font-size: 12px;
      color: #888;
      margin-top: 5px;
    }

    /* Username styling */
    .username {
      font-weight: bold;
      margin-bottom: 5px;
    }

    /* Content inside the bubble */
    .message-content {
      font-size: 14px;
    }

    /* Chat bubble styling */
    .chat-bubble {
      max-width: 70%;
      padding: 10px 15px;
      border-radius: 10px;
      word-wrap: break-word;
      position: relative; /* Required for pseudo-elements */
      box-shadow: 0 2px 5px rgba(0, 0, 0, 0.1);
    }

    .chat-message-left .chat-bubble {
      background-color: #e8f0fe; /* Bot message background */
    }

    .chat-message-right .chat-bubble {
      background-color: #d4edda; /* User message background */
    }

    /* triangle for left-aligned bubble */
    .chat-message-left .chat-bubble::after {
      content: "";
      position: absolute;
      top: 10px; /* Align with the top padding */
      left: -15px; /* Position outside the bubble */
      width: 0;
      height: 0;
      border-width: 10px; /* Increase the size of the triangle */
      border-style: solid;
      border-color: transparent #e8f0fe transparent transparent; /* Triangle pointing left */
    }

    /* triangle for right-aligned bubble */
    .chat-message-right .chat-bubble::after {
      content: "";
      position: absolute;
      top: 10px; /* Align with the top padding */
      right: -15px; /* Position outside the bubble */
      width: 0;
      height: 0;
      border-width: 10px; /* Increase the size of the triangle */
      border-style: solid;
      border-color: transparent transparent transparent #d4edda; /* Triangle pointing right */
    }
    
</style>

In [ ]:
# Stores a history of user questions and bot responses.
chat_history = []

# A text event handler for processing user input, displaying it in a chat-like interface,
# and retrieving a response from an agent. The function manages the chat's visual updates,
# input resetting, loading animations, and error handling.

def text_eventhandler(*args):
    
    # resetting the text input to an empty value.
    if args[0]["new"] == "":
        return

    # Show a loading animation and update the input field placeholder.
    loading_bar.layout.display = "block"
    in_text.placeholder = "One moment, please..."

    # Capture the user's question.
    question = args[0]["new"]

    # Reset the input field for new entries.
    args[0]["owner"].value = ""

    # Format the user's question as an HTML block for the chat UI.
    q = (f"""
            <!-- User Message -->
            <div class="chat-message chat-message-right">
                <div class="chat-meta">
                    <span class="icon-person"></span>
                    <div class="datetime">{datetime.now().strftime("%H:%M:%S")}</div>
                </div>
                <div class="chat-bubble">
                    <div class="username">You</div>
                    <div class="message-content">{question}</div>
                </div>
            </div>
        """
    )

    # Append the formatted question to the output display.
    output.append_display_data(HTML(q))

    try:
        # Stream responses from the React Agent step by step by calling it
        step_answers = []
        for step in agent_executor.stream(
                {"messages": [{"role": "user", "content": question}]},
                stream_mode="values",
                config=config
        ):
            step_answers.append(step["messages"][-1].content)
            
        # Retrieve the final answer and add it to the chat history.
        answer = step_answers[-1]
        chat_history.append((question, answer))
    except Exception as e:
        # Handle exceptions by displaying an error message.
        answer = "<b>Error:</b> " + str(e)

    # Format the bot's response for the chat UI, escaping special characters (e.g., $).
    answer_formatted = markdown.markdown(answer.replace("$", r"\$"))
    a = (f"""        
        <!-- Bot Message -->
        <div class="chat-message chat-message-left">
            <div class="chat-meta">
                <span class="icon-robot"></span>
                <div class="datetime">{datetime.now().strftime("%H:%M:%S")}</div>
            </div>
            <div class="chat-bubble">
                <div class="username">LLM</div>
                <div class="message-content">{answer_formatted}</div>
            </div>
        </div>        
        """
    )

    # Hide the loading animation and reset the input placeholder text.
    loading_bar.layout.display = "none"
    in_text.placeholder = "Ask a question about your data and press Enter"

    # Append the formatted bot response to the output display.
    output.append_display_data(HTML(a))

In [ ]:
# Define a Text widget for user input
in_text = widgets.Text(
    description='Question:',  # Label for the input field
    placeholder="Ask a question about your data and press Enter",  # Guide text for the user
    layout=widgets.Layout(height="30px",width='100%') # Set height and full width
)

# Disable continuous updates to avoid excessive event triggering
in_text.continuous_update = False

# Attach an observer to handle changes in the text input
in_text.observe(text_eventhandler, "value")

# Output widget to display responses and formatted chat messages
output = widgets.Output()

# Base64-encoded GIF for loading spinner
base64_gif = """
R0lGODlhMAAwAPcAAAAAABMTExUVFRsbGx0dHSYmJikpKS8vLzAwMDc3Nz4+PkJCQkRERElJSVBQUFdXV1hYWFxcXGNjY2RkZGhoaGxsbHFxcXZ2dnl5eX9/f4GBgYaGhoiIiI6OjpKSkpaWlpubm56enqKioqWlpampqa6urrCwsLe3t7q6ur6+vsHBwcfHx8vLy8zMzNLS0tXV1dnZ2dzc3OHh4eXl5erq6u7u7vLy8vf39/n5+f///wEBAQQEBA4ODhkZGSEhIS0tLTk5OUNDQ0pKSk1NTV9fX2lpaXBwcHd3d35+foKCgoSEhIuLi4yMjJGRkZWVlZ2dnaSkpKysrLOzs7u7u7y8vMPDw8bGxsnJydvb293d3eLi4ubm5uvr6+zs7Pb29gYGBg8PDyAgICcnJzU1NTs7O0ZGRkxMTFRUVFpaWmFhYWVlZWtra21tbXNzc3V1dXh4eIeHh4qKipCQkJSUlJiYmJycnKampqqqqrW1tcTExMrKys7OztPT09fX19jY2Ojo6PPz8/r6+hwcHCUlJTQ0NDg4OEFBQU9PT11dXWBgYGZmZm9vb3Jycnp6en19fYCAgIWFhaurq8DAwMjIyM3NzdHR0dTU1ODg4OTk5Onp6fDw8PX19fv7+xgYGB8fHz8/P0VFRVZWVl5eXmpqanR0dImJiaCgoKenp6+vr9/f3+fn5+3t7fHx8QUFBQgICBYWFioqKlVVVWJiYo+Pj5eXl6ioqLa2trm5udbW1vT09C4uLkdHR1FRUVtbW3x8fJmZmcXFxc/Pz42Njb+/v+/v7/j4+EtLS5qamri4uL29vdDQ0N7e3jIyMpOTk6Ojo7GxscLCwisrK1NTU1lZWW5ubkhISAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAACH/C05FVFNDQVBFMi4wAwEAAAAh/i1NYWRlIGJ5IEtyYXNpbWlyYSBOZWpjaGV2YSAod3d3LmxvYWRpbmZvLm5ldCkAIfkEAAoA/wAsAAAAADAAMAAABv/AnHBILBqPyKRySXyNSC+mdFqEAAARqpaIux0dVwduq2VJLN7iI3ys0cZkosogIJSKODBAXLzJYjJpcTkuCAIBDTRceg5GNDGAcIM5GwKWHkWMkjk2kDI1k0MzCwEBCTBEeg9cM5AzoUQjAwECF5KaQzWQMYKwNhClBStDjEM4fzGKZCxRRioFpRA2OXlsQrqAvUM300gsCgofr0UWhwMjQhgHBxhjfpCgeDMtLtpCOBYG+g4lvS8JAQZoEHKjRg042GZsylHjBYuHMY7gyHBAn4EDE1ZI8tCAhL1tNLoJsQGDxYoVEJHcOPHAooEEGSLmKKjlWIuHKF/ES0IjxAL/lwxCfFRCwwVKlC4UTomxIYFFaVtKomzBi8yKCetMkKnxEIZIMjdKdBi6ZIYyWAthSZGUVu0RGRsyyJ07V0SoGC3yutCrN40KcIADK6hAlgmLE4hNIF58QlmKBYIDV2g75bBixouVydCAAUOGzp87h6AsBQa9vfTy0uuFA86Y1m5jyyaDQwUJ0kpexMC95AWHBw9YkJlBYoSKs1RmhJDgoIGDDIWN1BZBvUSLr0psmKDgoLuDCSZ4G4FhgrqIESZeFMbBAsOD7g0ifJBxT7wkGyxImB+Bgr7EEA8418ADGrhARAodtKCEDNYRQYNt+wl3RAfNOWBBCr3MkMEEFZxg3YwkLXjQQQg7URPDCSNQN8wRMEggwQjICUECBRNQoIIQKYAAQgpCvOABBx2ksNANLpRQQolFuCBTETBYQOMHaYxwwQV2UVMCkPO1MY4WN3wwwQQWNJPDCJ2hI4QMH3TQQXixsVDBlyNIIiUGZuKopgdihmLDBjVisOWYGFxQJ0MhADkCdnGcQCMFHsZyAQZVDhEikCtOIsMFNXKAHZmQ9kFCBxyAEGNUmFYgIREiTDmoEDCICMKfccQAgghpiRDoqtSkcAKsk7RlK51IiAcLCZ2RMJsWRbkw6rHMFhEEACH5BAAKAP8ALAAAAAAwADAAAAf/gDmCg4SFhoeIiYqLhFhRUViMkpOFEwICE5SahDg4hjgSAQJEh16em4ctRklehkQBAaSFXhMPVaiFVwoGPyeFOK+xp4MkOzoCVLiDL7sGEF2cwbKDW0A6Oj0tyoNOBt5PhUQCwoRL1zpI29QO3gxZhNLDLz7XP1rqg1E/3kmDwLDTcBS5tgMcPkG0vCW4MkjaICoBrgmxgcrFO0NWEnib0OofORtDrvGYcqhTIhcOHIjgYgiJtx9RcuBQEiSIEkFPjOnIZMiGFi3DCiVRQFTClFaDsDDg1UQQDhs2kB4x1uPFrC1ZsrL8tCQIUQVBMLgY9uSBFKSGvEABwoSQFy5Z/7NqgVZqygSvRIU0uSeTrqIuSHF00RI3yxa0iLqIePBVwYMoQSX5LKyF4qQsTIR8NYJYEla5XSIzwnHFSBAGtzZ5IcylsyYvJ564lmz5oO3buAttabKEie/fS5bE3LYFi/Hjx7MgtZKyefMhQzCIpvTiipUr2LNjp8vcuXck0ydVt649O90tTIIrUbKEfXsS4T0jn6+ck0x/8XPr34/Dyon8iRimDhZOFFGBC6hwMcUULfhFCRckGFHEBEUwAeAvLUhxwglUYDFbXRgUMeEEGExxYSFaULHhhlUApQgOLSwh4gQTGCECXyYtMowNL6i44hVcTIcDCRXQOEEFTVg1SPAVT0SSyBZVKClIFy1MIYWGUzhpyBM0FpGEFYhxscQRSKTmiTwkiCBFbTJt4d+GCB6CxRFHROGgTFLQiYQ2OVxBAgkM5ZAFFCKIECgnWVBBBZuFvMBXIVkkcQQGIpwiRXBSOFVFoSRsVYgNd0qCwxMYHJHERTlcykSmgkBYaBUnStICEhhgIMUwly7BqiBXFAoFqurY0ASdS3iaam+75mCDFIWe8KEmVJSKQWqD5JpsDi8QCoWUymwxJgZOMGrtL1QUaqc6WShBJreCjItimlEYi4sWUNxqiLu5WCHvNtPhu98iJ/hG0r+MdGFcqAQTHAgAIfkEAAoA/wAsAAAAADAAMAAACP8AcwgcSLCgwYMIEypcSDALHjxZGEqcWNCNAQNvKGokGCjQQTYX2Ry84XHjQT4a5JQk2CakwRtu1OQxWXCPAwVlqhQMBNJAm5UCoxAIcEAnTYF+bipYU4NjSwNsgP5pEIAon6MD6yjYeqdgzzYF5QgIIAAO1oF/0mxFI4NgT5ED/YypuqDtWYFSFmyVMzDQ06gCA7kZO8DO3YGA2mw1c1Xg24FVxIxFA8hkH7sF9TTY+uZGDr8XweYAhKaqGCoH96BG2CeNmihNOTLZugCFQCYOHDARaGcAWdEEZ2QYIMCoQTlmcrep4nlgljM4RQQGBKi5Bt9j+hAEVAcBgO9ngAb/pnMmt4MzcLQPtMOmiviBN6KU4RuYSoMv3wF8UdN8ZxU35jkQAR0zCHRDZQvVUFIfaoCRHwBk3PEeQTVEoUaAa+AxYUI3xEHAg2HE8cdEM8yBRm5mZNCfRDWQkR8Ya6inEUoOoKGHSXZ88UUDVGzI0A0oSGgSIG/UseJhG/k4kZJIolUHHXQ8CeWUGmIFyB9YZvlHDVuWpMcaa6ihRphgihkHkwr9kcWabLbZ3B5hihnnmGowgWZCM7SpZxYIzkDHHHP8CeigUpzFpZaIirfSnU026ihHexi30QyxHZVFHW9k4IdJNeyhhx8IalSDFHC8YWodjA7Uhx6s7iEDozdU/8HEG26YGoekE/3hKat68FGgQoHwMYeptGogxYiBaXRDFp7mwSqoCAUiRQbEZiBCRAPtIQW2CP2hB2aj+cErq+ASZAexcuwBVA11MJFuXytlgQIezBX0x6qscltQFnDEQUWoA1HBhLvq8YECCurNMC8Km+40wx57HNnQrwXJMMfAUngUSBUiiGBUIHs8REWl2wG8pBRMxDEHZhx7XFINVOCBgrpN9iHHwJK2LGkfD6FA8Vk32DFwHSTrTNANMeOhR6oJ6THwuwQZ3VDP+tL0Bx0D33Gk1H3p8VAVJm8kA9ZyVJ0DFR3jmoPCUox81x94rFYQx3WonYMffIR91IRcPxHKUB522DGT3xIBsqbehCceEAAh+QQACgD/ACwAAAAAMAAwAAAI/wBzCBxIsKDBgwgTKlxI8BIVSZcYSpxIkNMjBQo4UNxYkNNBRxgfHdzkkeNBLB3qlBzIqRFGRwY5OVpEyWRBS4kcPJjU0aUCmAXxIDCggKdNgVkQOXDgSFNFn0AHdkFjgKilowOhLHUgpaBPkQTrVDUwB+vATIuWrsHE8itBLAyqOmBrViCVpYfqEITK8lHVH13rCtz0aCmiqzlahhy4olBVRU45YqFbsBKapZA8KlYAdtOaqoRWHKwkaWVBLG7c4IlMcI6DQw8kCQSxaI0IgSV+VI06EBOHHz9EHwShqDikSaYvKYIdSSAnkiU76GaAheAmKIYECAigyLRzKGuKK/9aMwfLyhKOkCPcJOWBXueS0AgKEECAIEbenU+CFL44IyiZOLcJQ5oMmAMWjAxCn3YMSGEgQprg0Yh4azQyRX4KceIBIdvVR4gHAUqECRSMiNcBhgl1IUSHgzBSHUeWeLAGTSZFIoggaKyAIkObSCLFjgkRJgJrghVpJEeaJaakaV1EIgIUUD4JhQgiUIFVS4dspaUDaCBWSSNugNnImGG6AQKQCnWBgA5stulmczl8KWaYYjZy5lFquqmnDnA2KSWUU05p5VFY4rVllxkeyUlJSaJ5ZF2cWEKJowcVaBYmUngwRxYmbXLJJZk8SJEmVMzBQQcclEApQZlk4eolXVD/tMkkdXRgqwd11MSRJp++egmRCGURiQeocjCHJLEmtqpzXVziahagiloQFR5wcKoHUkQ0EBZUUFbpZBVh8iy0yRqEx6kdQIHYQJpIIUIk6yopECaUTFKJtJuI62q5BWECAgiTAJsDJYBymkMWK6xgcBf1UqJtRbxesiOoB2XipAilCUQJHnjoeuAk9krr3LIsSUJlJCHGybHHmtQ7yYtFXjKlCB6r3HFDIFPCL1ab4EGlFERujEcl1lUCcrxYWRIo0pWs3C/Ik3hrUxclUHlhZU5XhEW995qVSdWRPDyQ0EQX1AXIlQjMUSYrGFUQ2Qc5KzKho3Fc9qMTNY0H0ngrCrRJJqH2LXhCAQEAIfkEAAoA/wAsAAAAADAAMAAACP8AcwgcSLCgwYMIEypcSFBVlTyqGEqcSJBTBwdmPFDcWJDTwVIOHHQ4yMkjx4Op6pwySXBDyFIGvZTS8OJkQRikFFXY0xGkA5gFpxj6ZIaPzYGXcioqxaqiS5EFVyn6ZCgUjKMDTShSNGpKQZ9AB5r6RLYO1oGrNGx1FFEgJ58jB6ZyQFYRjbMDq4zaGokgSDMdTFokC8orXoFePGy1cDUHp6dxc7BoQPZNU46p2hZ8YWHrBy8C4SK2QLYBT4MvWLAsmGpDqRSXB3IytXcUC4GR3rzpm8OEoaEaC9L4QPb2wVO633jYs1rVG50m3HopKbAOqE+hUhFkhcqBge8VVrv/NeEouSNTqVie6MBHvOwqFXg7zqPowHcDCRy5d8znQ/I3GqByl2OgLTSdQKloUMh9BoRyQoEIsVJFB/+Vksd+CXFShyEMGlLHKhPRYIIGydWBIUKriHJfAhpoh5kpjtB0EioHHKCIakd5sceFJ7HSASoQHibkkBx5ZKRjSKJ1gglLMumkCcbZ5MUGolRppZWKNAZDBx2UUkqXXX4ZyYkLsQJKAGimKQCaAqAi0JZfesllmPKdtIoha66ZJptu5rDKFCYw2WSgJ+SB1WNXJpqlQmRuZOSjbhEpqUGcpFJTj2/UEdtJNFRxyimaUWTKF1+YkUKjBrGyRySmtJoCR6t8/wLArAGMcilDXrxgwimtnmLCrRPJ5Mmss3pSyoAIcXLJFLzyGgkLsaFK0AuK8EAsAIVEEiRBe/DaaxXI5pAKC+HGpEq0KTTwBbFfKLKtQFX0ekJ626VwwhQupnpJKpesxkodBxAbyn40oIIKH+++cMK9bV3ywgttsZLKxCAWdIkGnXRSRUI0VCycvSeclgMMeeSRryoTX/JuDnucehILC6fg8bgsNJaDF/umUu5ZqgB6gs0js1AzQaukvPJJXuSxcBWbwsCCyRXtC4Mq0i6UysInXHKT0PkKVPTEm9rEir1Qiud0HkALhDK/VaNYhQlT7Oz00AVJzO/RFK3CR9pvPhndNVo0tG0TyXRPKhHNfxue4Sqr4K244QEBACH5BAAKAP8ALAAAAAAwADAAAAj/AHMIHEiwoMGDCBMqXEhwBgsWNBhKnFjwiRo1pihqLMjpIK2LdA7m6rjxoJYRJkgS/KgmZMFctGZhKVkwy4Y3jnBxZOmS4IpYh2TppClwxs03dDQV/Eihp8BVRxw4UKOF6MAUb7KuIMiJliw1TwqikuqgltWBmjxknRVRYFeQBLXIknpk1dmBlBxlNbHyYtiBtKTGUnF3ICdTR45oyAL4a08XaKRuyFVyRtuaGrI+6fgWrMBcGqRGGFoQF6WEM2jRWUFZbFZHp3OYWLKEb44UQB04FUiDjlQXCG3RnjUCl8ocNJbgJJyDk/OBtWI5oFB1YC4TsgwpULABYQoPS2aF/0dVXaCKJzMRcmLhyJZhFm20bzfk4bhhLLXEi6eVwm5z+yKRlMUSQmyngCEUqAAgQblQ8oR44dFByYIJcTKCAwYqgEYtSkm0Sgq0hDcLKhQilMsi8h3iQXkUzWDCLB4wtpEKZRjyBnBEcWJaiRWacktrhQUpZEmcNefWcwJpsoIKS6rApJMqkEbkLItUaWUbbSxyhIwnmWLKCF6G6aNVmjgAy5kFoHkmLO7l0KWXYIp5C5lmrmnnmW0qCeWTT+JIEydUWiloG1sOuRCSziFp6KKGzSDjRppoMAKQJa1CyS23XEYRKoIIgoaCkGKRgi2ksgCpEAGkWsARUirESRYqkP9KqgosSgQTAq+kGkACHmhqECcOyXpLClgAyeNTrWHRRgG6viKECZQShMUtwlLiH2+4XGtQLiMksIRhKqAhiK6CtLGgC6TessIMxzXIAiUzIPRGKwD44GcOmoxgSK4ByLLgKk5mAaAWD7Hg3yozzODfE/QCoIZ9Rh1wwFYIrdJhQZaysEJ6yGWRRVuaHAIAAGCkcJALzG2ExUOUXEyDx5elAMbIQlx81yoas8Diyx8bpsbIrfx1FycurMCCC5TyrCkuPoyMQK00zWA0RAU52jNBS4wMgCN35eKCxsYVpHTVQIzcQ2xEaULJQ9ryBrNBtbgCwCsmn5VLFlB3fDWDFAwUxihBY297bGGB/31oLiMZrnhBAQEAIfkEAAoA/wAsAAAAADAAMAAACP8AcwgcSLCgwYMIEypcSDCTCxeZGEqcWPDOmzd3KGosyOmgnQtv7Bzk1HHjQVW2qJQk+PGCyII3RPxKZbKgql9MmtAsaOeiCIMs2Ci64KfmwEw4mdy5UVDExZcDWUFSNFSV0YEsmGhlQZDTxzc/CdqiusbW1ah2tIqowfIpQVVvqEJidXbgiyZaqbAEKaIkJxFU2QCrO5CTCa1OLg38CvWFBapOVlLMxNbgJSdaTXT06jYHpyZULbw4mMpFwkwlSrhgWpCK1iajc1D59UtvDhVrqEIdWEOEBAlFDwITIcKOrVSSe+cMVnilCaG+rA68QYUNrwa8miBkYYd4cRURBwb/K7FzZDAmtgW60PCA1/UHvyQTvISiO/E7LOh6ln+QdY7LETSA3QNvsMBfVy+Y4J0dJvhxYEKclCCBe+4pYoJ+DLESzB3epTfRDb5gx0sEv0inUSYq2HGHYhux0B4TsdXESSoxahShCv4RpuOOJpHk2Y+S3eBCMEMGY2SR5dUUAkhv+HKRk29owGImKJhggi1YYnklMA8ydAMbCoQp5gJhLmAbSlnacqWatgxm1JdixlmmbUIaeeSdSW70ly++aNCnn3wywSKPhBZaVyYmanQDEyVgaBIrfgTDQmUamaCLLooYuNENqUjKAjDBUVRDLwaUmoAGeUKoigufAsMCRJuG/7BLqaXuEkJ4CdXwAgutBnNJlwfVwJofGiRAqwEPoJAjQanw6ioLqTjKiirLEnTDHbtoJxAnwCiiC60I+HJgs66+UINknFySSrQC3cDKuQJpMEAACdR4gwkN0GrBgaw8pAp/mazLLidvXHqBQHbMK4AFBqniRJhcIcRKtTncoG4q4XHCCwAA8CIQK70EEIAYKhy0K7AIBZzKrwNt3HFJKoghci+OnsXKupdQqjHHHg9kgQABDLDbWar4sfJKO3dMkB8JiLxAokbVILCjSfc8UBNAB8BEXemm4gfUVUuWSQMi68LcVRavvGzYBZVAgAC6lHwWJ5Qd5LLV01kggZuGehZ2d38oE9YLxxH0LdELdthRo+GM5xAQACH5BAAKAP8ALAAAAAAwADAAAAj/AHMIHEiwoMGDCBMqXEiQGAwYxBhKnFgQhTBhKChqLFjsoIklwkwc7LgRYSZgVw7iuSiSowk7l0oWzFRCBEyDJlga5JMBg5IsMgcSMyFCBAqSA3OGLGjjiRufM4IO5GPHJq6CSvEUlISh6zCpA3OhKGrCBsGcS1oKzLSkqxyzYAVeqiqCEkE8ILUmdeMmg924AotJKloi08CVS/TmyKKk6xOkFInBnRmpqCSSaFsWE9E1CVCDl2AkJCZpWBbIAq8UtfP5SqRIKXNQyvBUrVATfD/vxMMb2AzINohGuhoYqaSeSwwPFJxEkfPHB2Gg4I0HBaWIA2FIioqwGIwnkgji/5JTxLmiIpESZroynfcwXLmWM0Q6t4L5IksooeZ4SRJ1FJLEtBEKbtyHwTCTLZQLDMO0d8V+ChUjjHmM2KGcRsRQggIKF1JESQUVOKGbTJmMSFExeAADIWAstjgRSTBCVkwWD2VBIww3cidTMZEoscQSPgL5oxzcEXPFkUgmSdyOGTgwhANQRvkkMAIZmeSVS5ZUDAZRSjnEEKFQmcOMONqIY406yhQJSBe1CRKRLkq0Ypx0DmRDgic+YUJ8QeWSySWX8KmRJAww4IZ+GxVDzCU2ZpGmRLm4ocCkQixhYkLF2DBDo47iOV8koUw6aSgiYJdQLps2egkxJOXiqUE28P95iRxDiBqEIigIWtCiqmYCmTCFiKArQcWYEMoTBFGCQRC2LgFhiTbOMCwuPejQihsCuWoDScL8YAADI4olgahJdDfDJZ4Wo4gO1iKbgxJBBKGEQCV4a0ASqBEjApRZcgQhCjywOwRcRAQQABHZKmKAAQmIWVAWf2lkgxDsBvBVDrkUfDBJVySwsCLDSvVEK+wWAaPGRCCVxMI/lMDiJT+w60OWKBOUBQMLO/CoTBmwq8MSxBb8CsIEPbGwAU7ERckr7BbSYQ4oQ0YMEQsr0O9GwzDdSnpBG0z0WQgYoEBsUkkSiiKeRl1QLhkwQjZYxYRcDBGvHDzSnC0qUrcieNcLmV0JJYjm9+AGBQQAIfkEAAoA/wAsAAAAADAAMAAACP8AcwgcSLCgwYMIEypcSBCQlmWAGEqcWHAFFBErKGqUKEmECEkHA21MCEhZn4OSLoI0mOzElpEFa7RE9rJgx48Gl8lZcqwmzByAJJ04sUIkwZsrB3qpxYTnn58Dlw09scymx4wEW8hhwuQK1IGBVpyQIsnLUY9Jc9R4whWK2a8C/yAbenIgUoLJuMqpCzdHoBZDkdUYuALtQC20mpYwqhHQ24KAWp5oYfQm1kBSuNLScnBLVYQllW1hPLDP1JrKkCFTJrDPTibJDEbesIHzwWVXcisbTNCLUGSfDV5J/IS3wL9yMCiHglBL7ucQCTp/mlBLiRYEl4lAohwDEimkCdb/gPH8SotljyUy/iMliRs3ymkpC2/wj7Lyyv7QXyhpSXcMS5Q1USBatLBCbjBsFMgTGMCXhBTUNYZbC8ZR1AcSSIgQHEw1RLiRJFfs19eIJKoH1nGkBfLHiiy2WOFIJdAioxwy1vhETV4so+OOPPo0UiBLKCLkkERil4MXD/HYI1RAEulkEUaq2OKUL2oUyAm0HHNMllweI4KHJYYp5k+AMBiRgrUkk56VyRjzxRcijHTFA7wkwdpGfRQBBgB8klGlQl4kwcugEBxjG0N/LOEDn3x6ssSaC12pCC9mUCpBCX8qVQsZjAIAhiJ1eZFpb0ZtcQwElFbqhiT7eaHIF4x+/2EMMozJYUwJkB4nCRvMlbYEnYM+cAx9gTzAKAJPnNnaGAF0ksRxgABilAigKPDAhr4ZQSkvTOwnSSedIOGjX0YIEIAnzAXCxKBMCITMAgoosER4NZQggQQJIpSMkTYVEEAAEJxphAEGsCGQFxjEawxWBS3DF0WAQPBvAQwPbIARRiljRrxG5AoTFJ0IIIAbRgVisREEyRHvAieMuMUCIo+Rr0AnSwdBvBGACdMS/wogR0E1E1RLvAo8AZcyB/xrjIcmE4yxeGzEy8vMMElygACelFBQ0xeHJ0m1vPD70woSdGxQ0AQFIoedIwaSKxsEG2xQICKWiEEBBmAw5kRSSQex4d6ADxQQACH5BAAKAP8ALAAAAAAwADAAAAj/AHMIHEiwoMGDCBMqXEhwE5ctmxhKnFgQFx48lShqlEjpYkaDxTYm3JQly8FKFymBpGSFi8iCmihdoVTDYEc8KgtqseMMlcuXAjdVunIFV0iCNz8OLIbCWc+aQAVyIXrl58CkBf04taM0ajFcRCtFHIgSJ8Eaz5ziGRtVYA2ZV7Qg9Yh0q8m2BLMQpaSJLF2pkZwOO6qxGGGCMYn6ufq32DCnkawS5CIXYTEtWvoa1LL3p94ri3Nk4eksZ0MrIEBsQcilZJYtmpcOpbRa4GFcgZ/FzvHVTocOHPAgrKHFdRYubHNwwQUV4ZZhuAhuQdWMA/Bmw0ZuMa6lxmGGhGtA/5vDwXqHSFm+G9S03XV3kZSe/Lb+hFJyhcWIu65NsRgq83MM0xxFDmF2n0RZNNPMM/y9tMluGhWlHl4UWmYbb7xN+NKEhOGCBi8ghhhiIwdS9BhPKDpjhx2RCRSJDjDGKCMzAxYGQiMX4Ihjjjl+ZIeMQOpAI1DFgMCjjhfk2MhHHooo4iGNaCgRNE5tpSJkkhmGYYYVdumlSJrYkUSJCxWDBzRkTomGIIJEAt8iozQT3UZ+XDBIAHgKUWOZzUzgZxt2NKgQF80QIgCeAhAyR5oHOdbIKH5O0AgeezaECigCHCrAIG2E9iBDmxzFhR1tRDqKEldweIEgmQYgyAPQEP/2xAPPkFnMFY6gQpAfcywyAaSjONPoBIgaYsdufoACywEd2BbqUZE8wMsEldl2hRKQTgDChFYccAAHguaQBCyDHKBrDs4sssgTAkHzwCGHzPFdDXjkeNdB0HQ1kBWEwALLBGM5ooACUfLGAS+HoKGvQFuEppEmE/hbyBUDCUzwQLhEAOKYXaLCjL9JEJbEwI0Q9ESI2VG4BS/+gnJvDhYXzPAEh/CyiGRAzeEvLOwSNPLFBOGBMC924IWLAv4+gLPFjhymSSMgRvCySFYgfYBwBcX83RXSprHwRlcswnHWJIMEQgcOt6WlQTE3+iVCHAwc8tsTaTHMMNXSrbdBAQEAIfkEAAoA/wAsAAAAADAAMAAACP8AcwgcSLCgwYMIEypcSPDGqlWcGEqcWDDLlStZKGqUaPEKlo0bOWXKdBDLFSsfDWJRZgNkwRtasmi5ofJkSoKZUOBRscrlQE4xs5AsaNJjQU5X8OBJ0dKnQBtZovYkWPSmQC1KUWR0KpDTlqhaIg6s2lCFUis0uT6NmmWqQLJjleLZohYn2LQ54OawkUIKnmBiNaYIdhBoVLpvL95UpjSFW4Krhh5U0amTBi0GV7FNu8WSJcRbdOKxZPCGshIlHv8MBaC1rhBNu37VonpgFp0q8ObglAUPFCjOrBy8oehLawBfGqQIbGOLboOZrmAemEkFcGfOoBAeXqvQcQA8FJH/psj8Si3s2FGEVZiplI/vPko9Z2hJCvYQUKRYCrzQkqIAxyVQm0KcqIBeLVfERlEKDXzxhTMgbVELFCpIBpINIbyhIEWWbKUWf3UlxMmIu0VEYogLYaGIKKKsyOKLkICo0RVS1FgjHjbiMZUUAfTo44+gDDhRLaUU2UGRpRzZQUol/OhkAKBsSF4tRxqJZAdLvuUiixO8KAok802ElI1k3uiWiSWSKCOKbLaJ0A0ldBDmQgUC5pQViugSjRQgWaJBBiF4SBEWGiRgQDTRTCMlgRm+8YYGUljIXghBGHBoNEGEMGdCVpTiqKMdqLDoQDfgMQ2iiCaQwU2bkipWJlJo//DpG07YaRAnGegZjQG6KGJFYLVQo8KauwXTAR4EZRFCBqQ4moEUMnLCCKoNlKAbFtOAkmlXuw2EBzWKvDFdV8E0IesbUCCkDBmFOCFpDk2wGwSfOUDxBinp5mAFuIo4AyJfkEAyrkFWKHNQMA2QAQopaXUgjTQx5nCDE4oowojBBn0F0g1vFFJIA1cMVIoZ0pQyFiMVN9GqRiiA4nETgZUijRkmDwRFxWsIV1cmiigciqAdkByxQJlkULEGQmrkjMug5Cvyw0MLlMIaFdPrVBbSeKyIpA6bAUlBNpRSMSmCgqRMKIWAgoJBI5dsUDBrUMOIVS4po0EpMsoMMYicQB7hRNk+nVhQ11/f6uZBTZDcweETbWGFFQMzLvlAAQEAIfkEAAoA/wAsAAAAADAAMAAACP8AcwgcSLCgwYMIEypcSLDYjRvFGEqcWPBPqlR/KGpseOOgRYwbN6oINaFjxYsZDWpJZTLkwGQEALiqZfBjSoJd9kyqBMjlwD2CAAAAclPgR0wGYUyatKelTyRCAXA4CZIgJp2TkPocqAWBUB8wCNpsWGmppYhbBz5pJZQC2hxjuS7d0yUtQUDVhAZINjBujhtYw4bMU+lgMh5Ch/SEi3JgqqWTFhe8URfhpB8/OGgdWIyC0FZPBHbBhKnyH8ipDBZLlUyF5IYTAgR4tcDO60oxWzVCiKlsJadw89gaXlh1GwKyAxCAoOItByC2EwKCUbRLpVvDbd2yhPCGiWqvkg//ciOYssYbMJJlv5V1IaZmhMLPJvTh7UQtKtarSGVfIQw3g4T3SjWVTVTMHtklYwlwDBWjAgQECELTRn/ccgtdWwFihwYMSpQKJv25FKJdCkX01ogkGpSKG9RQ04aLL7Y4S4cTWaLCjTjimMdithjg44+D/CjNaxvdIsKRSCJphxYC9fjjkz6GQiRFxSST5JVLCpRKIy3G2KKMNEpkY4457thQDvahmOKabCp0g5FhJnTgWVtV0sgCDKgQkhbNNGPCZhTxWc0nhLYRp2qozMLBLB8kU+BCgNQCAaGESmOHmgjtccwsis7yRFMlqkDBApRWw0FqaGIq0FtdJPNBp7PU/8LfQcU0wwClC7QxCUEmILFrQjA8oedAmJjQzKIcNMOXahpQGoEtr2lBgTShTGjiQCog0QgHRRVjiQiccnALQpVIM8QTRQl0zBDSSDNuDrZwwIEJAu2hbSP0TpbHMccAWtAe3BlkSQTscqguBRN8sKoIjbihAaoVMbnRDRu0C0FxORwzQcJopaKBG26IcChFI7GrsFoTUHCyQCY00ggSe6TYhRvsyiKxuhsfI9YsbjTSzJQh1WKuNKgUdAzCKwukgsuNLLuVFhOY68ajGW+c9F8f9KxZWpbIMkQowxKkMccFWYKEGxvc7BMMsxwT4thXo2lCliQWM6LGKtPaJkIipA8c2t4T/bHHHv4CbjhBAQEAOw==
"""

# Decode the Base64-encoded GIF into binary format
gif_bytes = base64.b64decode(base64_gif)

# Create a loading spinner using the decoded GIF
loading_bar = widgets.Image(
    value=gif_bytes,              # Binary data for the GIF
    format='gif',                 # Specify the image format as GIF
    width='30',                   # Set the width of the spinner (30px)
    height='30',                  # Set the height of the spinner (30px)
    layout={'display': 'None'}    # Initially hide the spinner using CSS display property
)

In [ ]:
title = widgets.HTML("""
    <h2>Demo: Interactive Q&A Chat UI.</h2>
""")
display(title)

# Add a horizontal line separator for visual organization
separator = widgets.HTML("<hr style='border: 1px solid #ccc; margin: 10px 0;'>")

# Display separator at the top
display(separator)

# User-Friendly Introduction
intro = widgets.HTML("""
    <p>
        Here we can test the solution by asking questions related to data profiling, data analytics, and data quality. 
        For example:
        <ul>
            <li>Counts of missing or empty fields.</li>
            <li>Detection of outliers.</li>
            <li>Basic descriptive statistics (e.g., min, max, mode, median, averages, distributions).</li>
        </ul>
        Simply type your question into the text box below and press <strong>Enter</strong>. 
        The system will process your question, design an appropriate SQL query, and respond in natural language.
    </p>
    <p><em>Shall we?</em></p>

""")

display(intro)
display(separator)

# Display output with chat_history


# Display the output box for chat history
display(
    widgets.HBox(
        [output],  # The output widget showing chat history
        layout=widgets.Layout(
            width="100%",                 # Full-width container
            display="inline-flex",        # Inline flexible layout
            flex_flow="column-reverse",   # Reverse the column flow to show the latest messages at the top
        ),
    )
)

# Add another separator below the output box
display(separator)

# Combine the loading spinner and text input into a single horizontal box
wBox = widgets.Box(
        children=[loading_bar, in_text],  # Place loading spinner and input field side by side
        layout=widgets.Layout(
            display="flex",               # Use a flexbox layout
            flex_flow="row"              # Arrange items in a row
        ),
)

# Display the input section for user questions
display(wBox)

# Add a final separator
display(separator)

### Example Questions
Below are some sample questions that were tested:

- What are the unique values in the column for country?
- What about currencies?
- How many records have empty values in the transaction type field?
- Exchange rate?
- How many records have empty values in the Calculate Tax?
- Are there any outliers in the transaction values?
- How many rows have null and non-null values in the transaction value column?
- How many records are missing or not missing in the fiscal year column?
- What is the total transaction value in the dataset?
- What is the total absolute transaction value in the dataset?
- What is the average exchange rate?
- How many documents are flagged as back-posted?
- What is the sum of transaction values grouped by currency?
- How many records are there for each transaction Type?
- What is the largest absolute transaction value?
- Can you give me basic statistics like average, median, and max for the transaction values?
- What is the 3rd quartile of the transaction values?
- 1st quartile?
- What is the average of the transaction values?
- What is the latest clearing date in the data?
- What is the first clearing date in the data?
- What time period is covered by the data based on the clearing dates?
- What time period in years is covered by the data based on the clearing dates?
- What time period is covered by the data based on the fiscal year column?
- What authorization groups are represented in the data?
- What are the exchange rates for USD currency?
- How many records are in the table?
- How many columns are there in the table?
- How many outliers in the transaction values?
- List all types of transactions available in the data.
- Show me all transactions with the highest value.
- Are there any unusual or extreme values in the transaction amounts?